In [1]:
from src.utils.synchronisation import NamedPipeServer

import win32pipe, win32file, pywintypes
import json, time

In [2]:
def task(message):
    print(message)
    return {"status": "ok"}

server = NamedPipeServer(
    callback=task,
    should_reply=False,
    )

In [ ]:
def send(msg, pipe_name: str = r"\\.\pipe\MatPy", timeout_ms: int = 1000):
    # Wait until the server side exists
    start = time.time()
    while True:
        try:
            pipe = win32file.CreateFile(
                pipe_name,
                win32file.GENERIC_WRITE | win32file.GENERIC_READ,
                0,                    # no sharing
                None,
                win32file.OPEN_EXISTING,
                0,
                None,
            )
            break
        except pywintypes.error as e:
            if e.winerror != 2 or (time.time() - start)*1000 > timeout_ms:
                raise
            time.sleep(0.05)          # retry until server is up

    try:
        win32file.WriteFile(pipe, json.dumps(msg).encode() + b"\n")
        hr, raw = win32file.ReadFile(pipe, 64 * 1024)     # optional reply
        return json.loads(raw.decode()) if raw else None
    finally:
        win32file.CloseHandle(pipe)


In [4]:
server.start()

In [5]:
send({"msg":"test"})

error: (536, 'WriteFile', 'Waiting for a process to open the other end of the pipe.')

In [6]:
send({"msg":"test"})

error: (231, 'CreateNamedPipe', 'All pipe instances are busy.')